In [1]:
import numpy as np
import scipy as sp
import sympy as smp
import matplotlib.pyplot as plt
from scipy.misc import derivative
from sympy.solvers import solve
from sympy import Symbol
from sympy import symbols, solve
import math
import funkcije

PRIMERJAVA RAZLIČNIH MODELOV ŽIVLJENJSKE DOBE

-Eksponentna porazdelitev ima gostoto
$$
f(x;\lambda) = 
\begin{cases}
\lambda e^{-\lambda x}, & x \ge 0, \\
0, & x < 0.
\end{cases}
$$
Tukaj mora biti $\lambda > 0$ in nam pove kako pogosto prihaja do okvar, in večja kot je hitreje pride do okvar.

-Weibullova porazdelitev ima gostoto
$$
f(x) = \alpha \beta x^{(\beta - 1)} e^{-\alpha x ^{\beta}}; x>0, \alpha>0, \beta>0.
$$
Parameter $\beta$ nam pove obliko porazdelitve, oziroma če se verjetnost okvare stroja s časom veča, manjša ali ostaja konstantna. Parameter $\alpha$ pa nam pove kako raztegnjena oziroma skoncentrirana je naša porazdelitev.

-Težkorepa poreazdelitev je taka, katere rep ni omejen eksponentno omejen. Poljudno to pomeni da se porazdelitev manjša bolj počasi kot pa pri eksponentni porazdelitvi. Tukaj imamo več različnih porazdelitev ampak mi se bomo skoncentrirali na log-normalno porazdelitev. Ta porazdelitev ima gostoto
$$
f(x) = \frac{1}{x \, \sigma \sqrt{2\pi}} 
\exp\left( -\,\frac{(\ln x - \mu)^2}{2\sigma^2} \right),
$$
kjer nam $\mu$ predstavlja pričakovano vrednost, $\sigma$ pa je standardna deviacija, $x$ pa mora biti pozitiven.

Sedaj pa bomo predpostavili da stroji imajo neko pričakovano življenjsko dobo in ko se pokvarijo takoj dobimo novega. Najprej bomo pogledali pričakovano število okvar v daljšem časovnem obdobju. Tukaj imamo prenoveitvene procese, saj ko se en proces konča se začne nov z isto porazdelitvijo kot prejšni.

Torej imamo za ekspnentno porazdelitvijo
$$
E[N_t] = \lambda t.
$$

Za ostali dve bomo pa našli pričakovano število okvar numerično in to s pomočjo prenovitvene enačbe. Ker velja pri prenovitvenih procesih $E[N_t] = m(t)$, kjer $m(t)$ izračunamo na sledeč način
$$
m(t) = F(t) + \int_{0}^{t} m(t-s) f(s) \, ds,
$$
kjer je $F$ kumulativna porazdelitvena funkcija naše slučanje spremenljivke, $f$ pa njena pripadajoča gostota.

Sedaj si bomo izmislili naše parametre da bo naloga bolj razumljiva. Za čas opazovanja bomo izbrali $T = 120$ mesecev. 

Za eksponentno porazdelitev bomo izbrali $\lambda = 0.5$

In [2]:
from scipy.stats import expon

lambda_rate = 0.25  
scale_exp = 1.0 / lambda_rate 
T = 20.0           
N = 10000          
dt = T / N


t_values = np.linspace(0, T, N + 1)

f_values = expon.pdf(t_values, scale=scale_exp)
F_values = expon.cdf(t_values, scale=scale_exp)


m_values = np.zeros(N + 1)
m_values[0] = 0


for i in range(1, N + 1):
    
    conv = dt * np.dot(m_values[1:i+1][::-1], f_values[1:i+1])

    m_values[i] = F_values[i] + conv


m_T_numerical = m_values[-1]

m_T_analytical = lambda_rate * T

print(f"Numerično izračunano N_T: {m_T_numerical:.4f}")
print(f"Analitično (referenčno) N_T: {m_T_analytical:.4f}")


Numerično izračunano N_T: 4.9894
Analitično (referenčno) N_T: 5.0000


In [12]:
from funkcije import monte_carlo_renewal_exponential

T_limit = 20.0
lambda_exp = 0.25        
N_runs = 10000

estimated_m_exp = monte_carlo_renewal_exponential(
    T=T_limit,
    lambda_rate=lambda_exp,
    N_simulations=N_runs
)

print(f"Monte Carlo za N_T: {estimated_m_exp:.4f}")


Monte Carlo za N_T: 5.0386


TOLEEEE JE ZA WEIBULLL

In [ ]:
from scipy.stats import weibull_min

beta_shape = 2 
alpha_scale = 4
T = 20         
N = 100000     
dt = T / N

t_values = np.linspace(0, T, N + 1)

f_values = weibull_min.pdf(t_values, beta_shape, scale=alpha_scale)
F_values = weibull_min.cdf(t_values, beta_shape, scale=alpha_scale)

m_values = np.zeros(N + 1)
m_values[0] = 0 

for i in range(1, N + 1):

    conv = dt * np.dot(m_values[1:i+1][::-1], f_values[1:i+1])

    m_values[i] = F_values[i] + conv

m_T_numerical = m_values[-1]

print(f"Numerično izračunano N_T: {m_T_numerical:.4f}")

Numerično izračunano m(T): 5.2788


In [13]:
T_limit = 20
beta_w = 2
alpha_w = 4
N_runs = 10000
from funkcije import monte_carlo_renewal_weibull

estimated_m = monte_carlo_renewal_weibull(
    T=T_limit,
    beta_shape=beta_w,
    alpha_scale=alpha_w,
    N_simulations=N_runs
)


print(f"Monte Carlo N_T: {estimated_m:.4f}")

Monte Carlo N_T: 5.2761


TOLEEE JE ZA LOGNORMALNOOO

In [ ]:
from scipy.stats import lognorm


sigma = 0.5
mu = np.log(4) - (sigma**2)/2
T = 20         
N = 50000        
dt = T / N

t_values = np.linspace(0, T, N + 1)

f_values = lognorm.pdf(t_values, s=sigma, scale=np.exp(mu))
F_values = lognorm.cdf(t_values, s=sigma, scale=np.exp(mu))

m_values = np.zeros(N + 1)
m_values[0] = 0


for i in range(1, N + 1):

    conv = dt * np.dot(m_values[1:i+1][::-1], f_values[1:i+1])

    m_values[i] = F_values[i] + conv

m_T_numerical = m_values[-1]

print(f"Numerično N_T = {m_T_numerical:.4f}")


Numerično m(T) = 4.6424


In [ ]:
from funkcije import monte_carlo_renewal_lognormal

T_limit = 20.0
sigma = 0.5
mu = np.log(4) - (sigma**2)/2
N_runs = 10000  

estimated_m = monte_carlo_renewal_lognormal(
    T=T_limit,
    mu=mu,
    sigma=sigma,
    N_simulations=N_runs
)

print(f"Monte Carlo N_T: {estimated_m:.4f}")


Estimated Expected Occurrences E[N(T)]: 4.6558


KONKRETEN PROBLEM

Imamo stroj, ki ima življensko dobo $T_0$, ki je eksponentno porazdeljana s parametrom 
$\lambda_0 = 0.25$ okvar na leto. Torej je njegova povprčna življenska doba štiri leta.

Stroju se po vsakem popravilu življenjska doba skrajša za $20\%$

$$
T_k = 0.8^k T_0.
$$
Kar pomeni da po prvem opravilu dobimo $\lambda_1 =\frac{\lambda_0}{0.8}$ in tako dalje...

Predpostavimo, da stroški okvare vključujejo tako strošek popravila kot tudi 
izpad dohodka. Strošek nakupa novega stroja se pojavi enkrat na cikel 
(cikel traja od nakupa novega stroja do njegove zamenjave).

Označimo strošek popravila s $C_p = 600$, izpad dohodka $C_d = 250$,
strošek nakupa novega stroja pa s $C_n = 8000$.


Ker gre za eksponentno porazdelitev, izračunamo pričakovan čas do naslednje okvare s pomočjo parametra $\lambda_i$ in dobimo $$E[T_i] = \frac{1}{\lambda_i},$$ kjer je $$\lambda_i = \frac{\lambda_0}{0.8^i}.$$

Torej dobimo pričakovan čas do okvare po i-tem popravilu
$$
E[T_i] = 4 * 0.8^i
$$

Tako dobimo pričakovan čas cikla
$$
E[čas cikla] = \sum_{i = 0}^{k-1} 4 * 0.8^i,
$$
kar je geometrijska vrsta in dobimo
$$
E[čas cikla] = 4 * \frac{1 - 0.8^k}{1- 0.8} = 20(1 - 0.8^k)
$$

Sedaj pa se lotimo stroškov v ciklu. Po $k$ popravilih po strošek cikla enak $8000 + 850k$. Torej lahko definiramo funkcijo stroškov
$$
C(k) = 8000 + 850k
$$


Sedaj pa lahko izračunamo povprečni strošek na časovno enoto in dobimo ven optimalno vrednost $k$, pri kateri se bo še splačalo stroj popraviti. Povprečni strošek na enoto časa bomo predstavili s pomočjo funkcije
$$
AC(k) = \frac{C(k)}{E[čas cikla]} = \frac{8000 + 850k}{20(1-0.8^k)}
$$

To funkcijo sedaj odvajamo in dobimo optimalno vrednost kjer $k$ doseže svoj minimum.

In [8]:
x = smp.symbols('x', real=True)

g = ((8000 + 850*x) / (20 * (1 - 0.8**x)))
dgdx = smp.diff(g, x)

sol = smp.nsolve(dgdx, 5)  
print(sol)


6.87140170058084


Torej vidimo da se splača stroj popraviti 6-krat, potem pa je naša najboljša poteza da nakupimo novega.

Sedaj bomo pa preverili to še s pomočjo Monte-Carlo simulacij.

In [ ]:
from funkcije import monte_carlo_time_to_k_failures

lambda0 = 0.25
C_r = 600   
C_n = 8000  
C_d = 250   
ks = range(1, 30)
avg_cost_per_year = []

for k in ks:
    
    E_time = monte_carlo_time_to_k_failures(k, lambda0, N_sim=10000)

    E_cost = C_n + k*(C_r + C_d)

    avg_cost_per_year.append(E_cost / E_time)

optimal_k = ks[int(np.argmin(avg_cost_per_year))]

print(f"Optimalni k = {optimal_k}")


Optimal k = 7


Sedaj pa poglejmo še kako bi izgledal primer če imamo Weibullovo porazdelitev. Weibullova porazdelitev ima gostoto
$$
f(x) = \alpha \beta x^{\beta - 1} e^{- \alpha x^{\beta}},
$$
kjer $\alpha$ predstavlja razteg te porazdelitve, parameter $\beta$ pa nam pove če se verjetnost okvare stroja skozi čas veča ali manjša. Za naš primer sem določil da bo $\alpha = nevemmmm$ in $\beta = 2$. 

Pričakovano vrednost pri tej porazdelitvi izračunamo tako 
$$
E[X] = \alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta})
$$
Ponovno bomo predpostavili da se po vsakem popravilu pričakovana življenska doba do naslednjega popravila zmanjša za 20%. Torej če velja
$$
E[T] = \alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta})
$$
iz tega sledi
$$
E[T_i] = \alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta}) 0.8^i.
$$
Iz tega ven dobimo čas cikla
$$
E[čas cikla] = \sum_{i = 0}^{k-1} T_i = \sum_{i = 0}^{k-1} \alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta}) 0.8^i = \alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta}) \frac{1- 0.8^k}{1-0.8}
$$
Stroški cikla so isti kot prej, torej strošek popravila s $C_p = 600$, izpad dohodka $C_d = 250$,
strošek nakupa novega stroja pa s $C_n = 8000$. Sedaj pa ponovno izračunamo povprečni strošek na časovno enoto in dobimo ven optimalno vrednost $k$, pri kateri se bo še splačalo stroj popraviti. Povprečni strošek na enoto časa bomo predstavili s pomočjo funkcije
$$
AC(k) = \frac{C(k)}{E[čas cikla]} = \frac{8000 + 850k}{\alpha^{-1/\beta}\Gamma(1+ \frac{1}{\beta}) \frac{1- 0.8^k}{1-0.8}}
$$
Tukaj lahko opazimo da lahko gledamo samo funkcijo 
$$
g(k) = \frac{8000 + 850k}{}
$$

TUKAJ VIDIMO DA KO BOMO ODVAJALI NAŠO FUNKCIJO BO ISTO KOT PRI EKSPONENTNI PORAZDELITVI, KER JE DEL Z SPREMENLJIVKO ENAK. TAKO DA LAHKO REČEMO DA ŠTEVILO PRI KATEREM SE NAM STROJ ŠE SPLAČA POPRAVITI SPLOH NO ODVISNO OD PORAZDELITVE, KAR JE ČE MALO POMISLIMO KAR LOGIČNO, SAJ NAM PORAZDELITEV GOVORI SAMO O ČASU MED POPRAVKI IN ČE GLEDAMO ŠTEVILO POPRAVIL NI ODVISNO OD ČASA DELOVANJA. Bomo pa še pogledali monte carlo simulacije, če res vrnejo iste rešitve.

In [ ]:
from funkcije import monte_carlo_time_to_k_failures_weibull

beta = 2
eta0 = 4


C_r = 600
C_n = 8000
C_d = 250

ks = range(1, 30)
avg_cost_per_year = []


for k in ks:
    E_time = monte_carlo_time_to_k_failures_weibull(k, beta, eta0, degradation=0.8, N_sim=10000)
    E_cost = C_n + k*(C_r + C_d)

    avg_cost_per_year.append(E_cost / E_time)

optimal_index = int(np.argmin(avg_cost_per_year))
optimal_k = ks[optimal_index]

print(f"Optimalmi k = {optimal_k}")


Optimal k = 7


Še simulacije za lognormalno. Tukaj je važno da smo faktor staranja 0.8 pomnožili s pričakovano vrednost porazdelitve in ne samo z mi kot smo navajeni.

In [ ]:
from funkcije import monte_carlo_time_to_k_failures_lognormal

C_r = 600
C_n = 8000
C_d = 250

mu = 4 
sigma = 0.5  

ks = range(1, 30)
avg_cost_per_year = []

for k in ks:
    E_time = monte_carlo_time_to_k_failures_lognormal(k, mu, sigma, degradation=0.8, N_sim=10000)
    E_cost = C_n + k*(C_r + C_d)
    
    avg_cost_per_year.append(E_cost / E_time)

optimal_index = int(np.argmin(avg_cost_per_year))
optimal_k = ks[optimal_index]
print(f"Optimalni k = {optimal_k}")


Optimal k = 7
